<a href="https://colab.research.google.com/github/Tclack88/Renewable-Energy/blob/master/Visualize_Power_Plants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

In [0]:
url = "https://raw.githubusercontent.com/Tclack88/Renewable-Energy/master/power_plants.csv"
data = pd.read_csv(url)
data = data.drop_duplicates()

In [0]:
def clean_desc(x):
  try:
    x_split = x.split('(')
    return x_split[0]
  except:
    return x

In [5]:
url2 = "https://www.eia.gov/electricity/data/eia860m/archive/xls/energy_source_codes_prime_mover.xlsx"

energy_codes = pd.read_excel(url2,header=1)

energy_codes['Fuel Type'] = energy_codes['Fuel Type'].ffill()
energy_codes['Energy Source Description'] = energy_codes['Energy Source Description'].apply(clean_desc)
energy_codes.Energy = energy_codes.Energy.ffill()
energy_codes = energy_codes.iloc[:,[1,2,6]].dropna()
energy_codes['color'] = ['black']*7+['purple']*8+['grey']*3+['forestgreen']*4+['brown']*4+['peru']*2+['gold','ghostwhite','orangered','dodgerblue','turquoise','dodgerblue','greenyellow','lavender','darksalmon','dimgrey','goldenrod','dimgrey']
energy_codes

,Fuel Type,Energy,Energy Source Description,color
3,Coal,ANT,Anthracite Coal,black
4,Coal,BIT,Bituminous Coal,black
5,Coal,LIG,Lignite Coal,black
6,Coal,SGC,Coal-Derived Synthesis Gas,black
7,Coal,SUB,Subbituminous Coal,black
8,Coal,WC,Waste/Other Coal,black
9,Coal,RC,Refined Coal,black
10,Petroleum Products,DFO,Distillate Fuel Oil,purple
11,Petroleum Products,JF,Jet Fuel,purple
12,Petroleum Products,KER,Kerosene,purple


In [6]:
def clean_prime_mover(x):
    if '(' in x:
      x_split = x.split('(')
      if ',' in x_split[0]:
        return (clean_prime_mover(x_split[0])).strip()
      else:
        return x_split[0].strip()
    elif ',' in x:
      x_split2 = x.split(',')
      return x_split2[-1]
    else:
      return x.strip()

prime_mover_codes = pd.read_excel(url2,header=1,sheet_name=1)
prime_mover_table = prime_mover_codes.iloc[:,[1,2]]
prime_mover_table['Prime Mover Description'] = prime_mover_table['Prime Mover Description'].apply(clean_prime_mover) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [0]:
# convert 2-3 letter Energy sources to human readable format
source_map = dict(zip(energy_codes.Energy,energy_codes['Energy Source Description']))
data.energy_source = data.energy_source.map(source_map)

prime_mover_map = dict(zip(prime_mover_table['Prime Mover Code'],prime_mover_table['Prime Mover Description']))
prime_mover_map['ST'] = 'steam'
data.prime_mover = data.prime_mover.map(prime_mover_map)

In [8]:
# map fuel colors to intuitive colors
color_map = dict(zip(energy_codes['Energy Source Description'],energy_codes.color))
# hover data
hover_name = 'plant_type'
hover_data = ['prime_mover','total_power']

# plot
fig = plt.figure(figsize=(10,30))
fig = px.scatter_mapbox(data_frame = data, lat = 'lat', lon = 'lon', size = 'total_power', color = 'energy_source',color_discrete_map = color_map,zoom=3.5,hover_data=hover_data,hover_name = hover_name,size_max = 25)
fig.update_layout(mapbox_style = "stamen-watercolor")
fig.show();
# "open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" or "stamen-watercolor"

<Figure size 720x2160 with 0 Axes>

In [9]:
# Save the human readable data format
data.to_csv('power_plants_human_readable.csv',index=False)
data.head()

,plant_type,energy_source,prime_mover,year_built,lat,lon,total_power,n_units
0,Conventional Hydroelectric,Pumping Energy for Reversible,Hydroelectric Turbine,1963,33.458665,-87.35682,53.9,1
1,Natural Gas Steam Turbine,Natural Gas,steam,1954,31.006900,-88.01030,306.2,2
3,Conventional Steam Coal,Bituminous Coal,steam,1969,31.006900,-88.01030,403.7,1
4,Conventional Steam Coal,Bituminous Coal,steam,1971,31.006900,-88.01030,788.8,1
5,Natural Gas Fired Combined Cycle,Natural Gas,Combined Cycle Combustion Turbine Part,2000,31.006900,-88.01030,680.4,4


In [0]:
energy_source_net_power = data.groupby('energy_source').total_power.sum().sort_values(ascending=False).reset_index()

In [11]:
energy_source_net_power

,energy_source,total_power
0,Natural Gas,541268.9
1,Nuclear,103632.2
2,Pumping Energy for Reversible,101758.5
3,Bituminous Coal,99999.2
4,Wind,98449.8
5,Subbituminous Coal,95516.9
6,Refined Coal,49758.2
7,Solar,33829.1
8,Distillate Fuel Oil,21079.2
9,Residual Fuel Oil,11252.3


In [14]:
#fig = plt.figure(figsize=(20,30))
subgroup = energy_source_net_power
fig = px.bar(subgroup[:15], x= 'total_power', y='energy_source',orientation = 'h',
             color = 'energy_source', color_discrete_map = color_map,
             title = 'Natural gas provides as much power to the U.S. as the next 6 leading energy sources')
fig.update_traces(marker_line_color='#000000', marker_line_width=1.5)
fig.show()